In [49]:
import numpy as np
import math
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib
%matplotlib inline

In [50]:
motions = ['nothing', 'unarmed-attack-r3', '2Hand-Axe-Attack1']
data_files = {motion: 'data/{}.txt'.format(motion) for motion in motions}

In [51]:
# Load training data
data = {motion: np.loadtxt(file_name) for motion, file_name in data_files.items()}

In [52]:
# Statistics about training data
for motion in motions:
    print("{}: ".format(motion))
    print('    shape: {}'.format(data[motion].shape))

nothing: 
    shape: (222, 1062)
unarmed-attack-r3: 
    shape: (157, 1086)
2Hand-Axe-Attack1: 
    shape: (147, 1044)


### Preprocess

In [53]:
def regularize_shape(data, num_samples):
    '''
    return num_samples of entries in data,
    (Note: data are groups of 6dof measurements)
    '''
    data = data.reshape(data.shape[0], -1, 6)
    length = data.shape[1]
    assert(num_samples <= length)
    # step = math.ceil(data.shape[1]//num_samples)
    result = []
    for data_instance in data:
        result_instance = []
        for i in range(num_samples):
            result_instance.append(data_instance[int(math.ceil(i*length/num_samples))])
        result.append(np.asarray(result_instance))
    return np.asarray(result)
NUM_SAMPLES = 64
data_regularized = {key: regularize_shape(val, NUM_SAMPLES) for key, val in data.items()}
for motion in motions:
    print("{}: ".format(motion))
    print('    regularized shape: {}'.format(data_regularized[motion].shape))

nothing: 
    regularized shape: (222, 64, 6)
unarmed-attack-r3: 
    regularized shape: (157, 64, 6)
2Hand-Axe-Attack1: 
    regularized shape: (147, 64, 6)


In [54]:
# Return X and y
def raw_data(data_dict, keys):
    lengths = [data_dict[key].shape[0] for key in keys]
    data_y = []
    for i in range(len(lengths)):
        data_y.extend([i]*lengths[i])
    data_y = np.asarray(data_y).reshape(-1, 1)
    data_X = np.concatenate([data_dict[key] for key in keys])
    assert(data_y.shape[0] == data_X.shape[0])
    return data_X, data_y
data_X, data_y = raw_data(data_regularized, motions)

In [55]:
def flatten(X):
    return X.reshape(X.shape[0], -1)
# data_X_flattened = flatten(data_X)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.1, random_state=0)

### Neural Network Model

In [57]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from keras.utils import to_categorical

In [58]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [59]:
# model = Sequential()
# model.add(Dense(8, input_shape=X_train.shape[1:], activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(8, input_shape=X_train.shape[1:], activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(len(motions), activation='softmax'))
# model.summary()
# model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [60]:
print(X_train.shape)

(473, 64, 6)


In [69]:
model = Sequential()
model.add(Conv1D(8, 6, padding='same', input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(len(motions), activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 64, 8)             296       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 32, 8)             0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 8)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 2056      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 27        
Total params: 2,379
Trainable params: 2,379
Non-trainable params: 0
_________________________________________________________________


In [71]:
validation_split = 0.1
history = model.fit(X_train, y_train_categorical, batch_size=X_train.shape[0], epochs=200, verbose=1,
          validation_split=validation_split, shuffle=True)

Train on 425 samples, validate on 48 samples
Epoch 1/200
425/425 [==============================] - 0s 65us/step - loss: 0.2142 - acc: 0.9106 - val_loss: 0.2067 - val_acc: 0.9375
Epoch 2/200
425/425 [==============================] - 0s 80us/step - loss: 0.2120 - acc: 0.9129 - val_loss: 0.2050 - val_acc: 0.9375
Epoch 3/200
425/425 [==============================] - 0s 69us/step - loss: 0.1887 - acc: 0.9482 - val_loss: 0.2005 - val_acc: 0.9375
Epoch 4/200
425/425 [==============================] - 0s 60us/step - loss: 0.1899 - acc: 0.9318 - val_loss: 0.2004 - val_acc: 0.9583
Epoch 5/200
425/425 [==============================] - 0s 56us/step - loss: 0.1523 - acc: 0.9506 - val_loss: 0.1987 - val_acc: 0.9375
Epoch 6/200
425/425 [==============================] - 0s 55us/step - loss: 0.1967 - acc: 0.9247 - val_loss: 0.1966 - val_acc: 0.9375
Epoch 7/200
425/425 [==============================] - 0s 74us/step - loss: 0.1682 - acc: 0.9412 - val_loss: 0.1906 - val_acc: 0.9583
Epoch 8/200
425/4

Epoch 62/200
425/425 [==============================] - 0s 56us/step - loss: 0.1180 - acc: 0.9647 - val_loss: 0.1949 - val_acc: 0.9375
Epoch 63/200
425/425 [==============================] - 0s 65us/step - loss: 0.1109 - acc: 0.9600 - val_loss: 0.2019 - val_acc: 0.9375
Epoch 64/200
425/425 [==============================] - 0s 63us/step - loss: 0.1477 - acc: 0.9459 - val_loss: 0.2002 - val_acc: 0.9375
Epoch 65/200
425/425 [==============================] - 0s 65us/step - loss: 0.1089 - acc: 0.9624 - val_loss: 0.1961 - val_acc: 0.9375
Epoch 66/200
425/425 [==============================] - 0s 72us/step - loss: 0.1639 - acc: 0.9482 - val_loss: 0.1949 - val_acc: 0.9375
Epoch 67/200
425/425 [==============================] - 0s 78us/step - loss: 0.1294 - acc: 0.9506 - val_loss: 0.1870 - val_acc: 0.9375
Epoch 68/200
425/425 [==============================] - 0s 86us/step - loss: 0.1142 - acc: 0.9624 - val_loss: 0.1851 - val_acc: 0.9375
Epoch 69/200
425/425 [==============================] -

Epoch 123/200
425/425 [==============================] - 0s 72us/step - loss: 0.0705 - acc: 0.9788 - val_loss: 0.1580 - val_acc: 0.9375
Epoch 124/200
425/425 [==============================] - 0s 81us/step - loss: 0.0820 - acc: 0.9765 - val_loss: 0.1568 - val_acc: 0.9375
Epoch 125/200
425/425 [==============================] - 0s 60us/step - loss: 0.1122 - acc: 0.9600 - val_loss: 0.1614 - val_acc: 0.9375
Epoch 126/200
425/425 [==============================] - 0s 70us/step - loss: 0.0757 - acc: 0.9835 - val_loss: 0.1617 - val_acc: 0.9375
Epoch 127/200
425/425 [==============================] - 0s 73us/step - loss: 0.0827 - acc: 0.9765 - val_loss: 0.1629 - val_acc: 0.9375
Epoch 128/200
425/425 [==============================] - 0s 72us/step - loss: 0.0963 - acc: 0.9624 - val_loss: 0.1677 - val_acc: 0.9375
Epoch 129/200
425/425 [==============================] - 0s 73us/step - loss: 0.0965 - acc: 0.9671 - val_loss: 0.1673 - val_acc: 0.9375
Epoch 130/200
425/425 [=========================

Epoch 184/200
425/425 [==============================] - 0s 64us/step - loss: 0.0756 - acc: 0.9765 - val_loss: 0.1699 - val_acc: 0.9583
Epoch 185/200
425/425 [==============================] - 0s 59us/step - loss: 0.0460 - acc: 0.9882 - val_loss: 0.1648 - val_acc: 0.9583
Epoch 186/200
425/425 [==============================] - 0s 73us/step - loss: 0.0737 - acc: 0.9741 - val_loss: 0.1546 - val_acc: 0.9792
Epoch 187/200
425/425 [==============================] - 0s 76us/step - loss: 0.0565 - acc: 0.9812 - val_loss: 0.1519 - val_acc: 0.9583
Epoch 188/200
425/425 [==============================] - 0s 70us/step - loss: 0.0496 - acc: 0.9882 - val_loss: 0.1519 - val_acc: 0.9792
Epoch 189/200
425/425 [==============================] - 0s 70us/step - loss: 0.0541 - acc: 0.9835 - val_loss: 0.1493 - val_acc: 0.9792
Epoch 190/200
425/425 [==============================] - 0s 65us/step - loss: 0.0709 - acc: 0.9765 - val_loss: 0.1591 - val_acc: 0.9792
Epoch 191/200
425/425 [=========================

In [72]:
# Temporary way of computing true positive rate / false positive rate
for i, word in enumerate(motions):
    pred = model.predict(X_test[(y_test == i).reshape(y_test.shape[0],), :], batch_size=X_test.shape[0])
    pred = pred.argmax(axis=-1)
    print("True '{}' rate: {}".format(word, sum(pred == i)/len(pred)))

True 'nothing' rate: 1.0
True 'unarmed-attack-r3' rate: 0.875
True '2Hand-Axe-Attack1' rate: 0.9230769230769231


In [73]:
result = model.evaluate(X_test, y_test_categorical)
print('')
print('test score: {}'.format(result[1]))

53/53 [==============================] - 0s 158us/step

test score: 0.9433962264150944


In [74]:
EXPORT_NN = True
if EXPORT_NN:
    model.save('model/keras/model_{}motions.h5'.format(len(motions)))